## 线性神经网络-linearnetworks
### Softmax回归
*本节小结：Softmax回归是一个多分类模型，使用交叉熵来衡量真实概率和预测概率的区别。*

### 图像分类数据集

In [2]:
import torch
import torchvision
from torch.utils import data
from torchvision import transforms
from d2l import torch as d2l

d2l.use_svg_display() # 封装函数，图像显示

OSError: [WinError 127] 找不到指定的程序。 Error loading "D:\conda\envs\A1\lib\site-packages\torch\lib\c10_cuda.dll" or one of its dependencies.

In [3]:
trans = transforms.ToTensor() # 将图像数据从PIL类型变换成32位浮点数格式
mnist_train = torchvision.datasets.FashionMNIST(  
    root="../data", train=True, transform=trans, download=True)  # 必须科学上网才能下载
mnist_test = torchvision.datasets.FashionMNIST(
    root="../data", train=False, transform=trans, download=True)

NameError: name 'transforms' is not defined

In [4]:
len(mnist_train), len(mnist_test) # 查看数量
mnist_train[0][0].shape # 查看输入图像的高度和宽度

NameError: name 'mnist_train' is not defined

In [6]:
def get_fashion_mnist_labels(labels):  #@save
    text_labels = ['t-shirt', 'trouser', 'pullover', 'dress', 'coat',
                   'sandal', 'shirt', 'sneaker', 'bag', 'ankle boot']
    return [text_labels[int(i)] for i in labels]

In [7]:
def show_images(imgs, num_rows, num_cols, titles=None, scale=1.5):  #@save 可视化样本
    figsize = (num_cols * scale, num_rows * scale)
    _, axes = d2l.plt.subplots(num_rows, num_cols, figsize=figsize)
    axes = axes.flatten()
    for i, (ax, img) in enumerate(zip(axes, imgs)):
        if torch.is_tensor(img):
            ax.imshow(img.numpy()) # 图片张量
        else:
            ax.imshow(img)
        ax.axes.get_xaxis().set_visible(False)
        ax.axes.get_yaxis().set_visible(False)
        if titles:
            ax.set_title(titles[i])
    return axes

In [8]:
X, y = next(iter(data.DataLoader(mnist_train, batch_size=18)))
show_images(X.reshape(18, 28, 28), 2, 9, titles=get_fashion_mnist_labels(y));

NameError: name 'data' is not defined

In [9]:
batch_size = 256
def get_dataloader_workers():  #@save
    return 4

train_iter = data.DataLoader(mnist_train, batch_size, shuffle=True,
                             num_workers=get_dataloader_workers())

timer = d2l.Timer()
for X, y in train_iter:
    continue
f'{timer.stop():.2f} sec'  # 读取训练数据所需的时间

NameError: name 'data' is not defined

In [10]:
def load_data_fashion_mnist(batch_size, resize=None):  #@save
    # 用于获取和读取Fashion-MNIST数据集，返回训练集和验证集的数据迭代器，参数resize用来将图像大小调整为另一种形状
    trans = [transforms.ToTensor()]
    if resize:
        trans.insert(0, transforms.Resize(resize))
    trans = transforms.Compose(trans)
    mnist_train = torchvision.datasets.FashionMNIST(
        root="../data", train=True, transform=trans, download=True)
    mnist_test = torchvision.datasets.FashionMNIST(
        root="../data", train=False, transform=trans, download=True)
    return (data.DataLoader(mnist_train, batch_size, shuffle=True,
                            num_workers=get_dataloader_workers()),
            data.DataLoader(mnist_test, batch_size, shuffle=False,
                            num_workers=get_dataloader_workers()))

In [11]:
train_iter, test_iter = load_data_fashion_mnist(32, resize=64)  # 调用上面定义的函数
for X, y in train_iter:
    print(X.shape, X.dtype, y.shape, y.dtype)
    break

NameError: name 'transforms' is not defined

### Softmax回归代码实现

In [13]:
import torch
from IPython import display
from d2l import torch as d2l

batch_size = 256 # 批量
train_iter, test_iter = d2l.load_data_fashion_mnist(batch_size)

OSError: [WinError 127] 找不到指定的程序。 Error loading "D:\conda\envs\A1\lib\site-packages\torch\lib\c10_cuda.dll" or one of its dependencies.

In [14]:
num_inputs = 784 # 28*28的图像
num_outputs = 10 # 10个类别

W = torch.normal(0, 0.01, size=(num_inputs, num_outputs), requires_grad=True) # 权重
b = torch.zeros(num_outputs, requires_grad=True) # 偏置

NameError: name 'torch' is not defined

In [15]:
Softmax表达式：
$$
\mathrm{softmax}(\mathbf{X})_{ij} = \frac{\exp(\mathbf{X}_{ij})}{\sum_k \exp(\mathbf{X}_{ik})}.
$$

SyntaxError: invalid character '：' (U+FF1A) (2208073562.py, line 1)

In [16]:
def softmax(X):
    X_exp = torch.exp(X)
    partition = X_exp.sum(1, keepdim=True)
    return X_exp / partition  # 广播机制

In [17]:
X = torch.normal(0, 1, (2, 5)) # 标准正态分布，均值为0，标准差为1，生成一个2*5的矩阵
X_prob = softmax(X) 
X_prob, X_prob.sum(1) # 按列相加

NameError: name 'torch' is not defined

In [18]:
def net(X):
    return softmax(torch.matmul(X.reshape((-1, W.shape[0])), W) + b) # 输入通过网络映射到输出

In [19]:
def cross_entropy(y_hat, y):
    return - torch.log(y_hat[range(len(y_hat)), y]) # 交叉熵损失函数

In [20]:
def accuracy(y_hat, y):  #@save 分类精度
    if len(y_hat.shape) > 1 and y_hat.shape[1] > 1:
        y_hat = y_hat.argmax(axis=1)
    cmp = y_hat.type(y.dtype) == y
    return float(cmp.type(y.dtype).sum())

In [21]:
def evaluate_accuracy(net, data_iter):  #@save 计算在指定数据集上模型的精度
    if isinstance(net, torch.nn.Module):
        net.eval()  # 将模型设置为评估模式
    metric = Accumulator(2)  # 正确预测数、预测总数
    with torch.no_grad():
        for X, y in data_iter:
            metric.add(accuracy(net(X), y), y.numel())
    return metric[0] / metric[1]

In [22]:
class Accumulator:  #@save 用于对多个变量进行累加
    def __init__(self, n):
        self.data = [0.0] * n

    def add(self, *args):
        self.data = [a + float(b) for a, b in zip(self.data, args)]

    def reset(self):
        self.data = [0.0] * len(self.data)

    def __getitem__(self, idx):
        return self.data[idx]

In [23]:
def train_epoch_ch3(net, train_iter, loss, updater):  #@save 训练模型
    # 将模型设置为训练模式
    if isinstance(net, torch.nn.Module):
        net.train()
    # 训练损失总和、训练准确度总和、样本数
    metric = Accumulator(3)
    for X, y in train_iter:
        # 计算梯度并更新参数
        y_hat = net(X)
        l = loss(y_hat, y)
        if isinstance(updater, torch.optim.Optimizer):
            # 使用PyTorch内置的优化器和损失函数
            updater.zero_grad()
            l.mean().backward()
            updater.step()
        else:
            # 使用定制的优化器和损失函数
            l.sum().backward()
            updater(X.shape[0])
        metric.add(float(l.sum()), accuracy(y_hat, y), y.numel())
    # 返回训练损失和训练精度
    return metric[0] / metric[2], metric[1] / metric[2]

In [24]:
class Animator:  #@save 在动画中绘制数据
    def __init__(self, xlabel=None, ylabel=None, legend=None, xlim=None,
                 ylim=None, xscale='linear', yscale='linear',
                 fmts=('-', 'm--', 'g-.', 'r:'), nrows=1, ncols=1,
                 figsize=(3.5, 2.5)):
        # 增量地绘制多条线
        if legend is None:
            legend = []
        d2l.use_svg_display()
        self.fig, self.axes = d2l.plt.subplots(nrows, ncols, figsize=figsize)
        if nrows * ncols == 1:
            self.axes = [self.axes, ]
        # 使用lambda函数捕获参数
        self.config_axes = lambda: d2l.set_axes(
            self.axes[0], xlabel, ylabel, xlim, ylim, xscale, yscale, legend)
        self.X, self.Y, self.fmts = None, None, fmts

    def add(self, x, y):
        # 向图表中添加多个数据点
        if not hasattr(y, "__len__"):
            y = [y]
        n = len(y)
        if not hasattr(x, "__len__"):
            x = [x] * n
        if not self.X:
            self.X = [[] for _ in range(n)]
        if not self.Y:
            self.Y = [[] for _ in range(n)]
        for i, (a, b) in enumerate(zip(x, y)):
            if a is not None and b is not None:
                self.X[i].append(a)
                self.Y[i].append(b)
        self.axes[0].cla()
        for x, y, fmt in zip(self.X, self.Y, self.fmts):
            self.axes[0].plot(x, y, fmt)
        self.config_axes()
        display.display(self.fig)
        display.clear_output(wait=True)

In [25]:
def train_ch3(net, train_iter, test_iter, loss, num_epochs, updater):  #@save
    animator = Animator(xlabel='epoch', xlim=[1, num_epochs], ylim=[0.3, 0.9],
                        legend=['train loss', 'train acc', 'test acc'])
    for epoch in range(num_epochs):
        train_metrics = train_epoch_ch3(net, train_iter, loss, updater)
        test_acc = evaluate_accuracy(net, test_iter)
        animator.add(epoch + 1, train_metrics + (test_acc,))
    train_loss, train_acc = train_metrics
    assert train_loss < 0.5, train_loss
    assert train_acc <= 1 and train_acc > 0.7, train_acc
    assert test_acc <= 1 and test_acc > 0.7, test_acc

In [26]:
lr = 0.1 # 学习率

def updater(batch_size):
    return d2l.sgd([W, b], lr, batch_size)

In [27]:
num_epochs = 5 # 迭代周期
train_ch3(net, train_iter, test_iter, cross_entropy, num_epochs, updater)

NameError: name 'train_iter' is not defined

In [28]:
def predict_ch3(net, test_iter, n=6):  #@save 预测
    for X, y in test_iter:
        break
    trues = d2l.get_fashion_mnist_labels(y)
    preds = d2l.get_fashion_mnist_labels(net(X).argmax(axis=1))
    titles = [true +'\n' + pred for true, pred in zip(trues, preds)]
    d2l.show_images(
        X[0:n].reshape((n, 28, 28)), 1, n, titles=titles[0:n])

predict_ch3(net, test_iter)

NameError: name 'test_iter' is not defined

### Softmax回归简介实现
使用Fashion-MNIST数据集，并保持批量大小为256

In [30]:
import torch
from torch import nn
from d2l import torch as d2l

batch_size = 256
train_iter, test_iter = d2l.load_data_fashion_mnist(batch_size)
net = nn.Sequential(nn.Flatten(), nn.Linear(784, 10))

def init_weights(m):
    if type(m) == nn.Linear:
        nn.init.normal_(m.weight, std=0.01)

net.apply(init_weights);
loss = nn.CrossEntropyLoss(reduction='none') 
trainer = torch.optim.SGD(net.parameters(), lr=0.1) # 使用学习率为0.1的小批量随机梯度下降作为优化算法
num_epochs = 10
d2l.train_ch3(net, train_iter, test_iter, loss, num_epochs, trainer)

OSError: [WinError 127] 找不到指定的程序。 Error loading "D:\conda\envs\A1\lib\site-packages\torch\lib\c10_cuda.dll" or one of its dependencies.